In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alopecia"
cohort = "GSE80342"

# Input paths
in_trait_dir = "../../input/GEO/Alopecia"
in_cohort_dir = "../../input/GEO/Alopecia/GSE80342"

# Output paths
out_data_file = "../../output/preprocess/Alopecia/GSE80342.csv"
out_gene_data_file = "../../output/preprocess/Alopecia/gene_data/GSE80342.csv"
out_clinical_data_file = "../../output/preprocess/Alopecia/clinical_data/GSE80342.csv"
json_path = "../../output/preprocess/Alopecia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Pilot open label clinical trial of oral ruxolitinib in patients with alopecia areata"
!Series_summary	"This goal of these studies were to examine gene expression profiles of skin from patients with alopecia areata undergoing treatment with oral ruxoltinib."
!Series_summary	"Microarray analysis was performed to assess changes in gene expression in affected scalp skin."
!Series_overall_design	"Twelve patients were recruited for this study.  Scalp skin biopsies were performed at baseline and at twelve weeks following the initiation of 20 mg BID ruxolitinib PO.  In addition, biopsies were taken prior to twelve weeks of treatment in some cases.  Biopsies from three healthy controls were also included in the dataset."
Sample Characteristics Dictionary:
{0: ['patientid: NC084', 'patientid: NC098', 'patientid: NC108', 'patientid: R01', 'patientid: R02', 'patientid: R03', 'patientid: R04', 'patientid: R05', 'patientid: R06', 'patientid: R07', 'patientid: R

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check for gene expression data availability
is_gene_available = True  # Based on the series description mentioning microarray analysis for gene expression

# Define conversion functions for clinical features
def convert_trait(value_str):
    """Convert alopecia status to binary (0: healthy control, 1: alopecia)"""
    if value_str is None:
        return None
    # Extract value after colon
    if ":" in value_str:
        value = value_str.split(":", 1)[1].strip()
    else:
        value = value_str.strip()
    
    if value.lower() == "healthy_control":
        return 0
    elif value.lower() in ["persistent_patchy", "severe_patchy", "totalis", "universalis"]:
        return 1
    return None

def convert_age(value_str):
    """Convert age to continuous value"""
    if value_str is None:
        return None
    # Extract value after colon
    if ":" in value_str:
        value = value_str.split(":", 1)[1].strip()
    else:
        value = value_str.strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value_str):
    """Convert gender to binary (0: female, 1: male)"""
    if value_str is None:
        return None
    # Extract value after colon
    if ":" in value_str:
        value = value_str.split(":", 1)[1].strip()
    else:
        value = value_str.strip()
    
    if value.upper() == "F":
        return 0
    elif value.upper() == "M":
        return 1
    return None

# Identify data availability and row indices
trait_row = 7  # 'aatype' in row 7 indicates alopecia status
age_row = 4    # 'agebaseline' in row 4 provides age data
gender_row = 3  # 'gender' in row 3 provides gender data

# Check if trait data is available
is_trait_available = trait_row is not None

# Initial metadata validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Create a proper DataFrame from the Sample Characteristics Dictionary
    sample_char_dict = {k: v for k, v in sorted(globals().get('Sample Characteristics Dictionary', {}).items())}
    clinical_data = pd.DataFrame(sample_char_dict)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{}
Clinical data saved to ../../output/preprocess/Alopecia/clinical_data/GSE80342.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown, these appear to be Affymetrix microarray probe IDs
# (e.g., '1007_s_at', '1053_at') rather than human gene symbols.
# These identifiers need to be mapped to gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation preview, we can observe that:
# - 'ID' column contains probe identifiers (e.g., '1007_s_at'), same as the gene expression data
# - 'Gene Symbol' column contains human gene symbols (e.g., 'DDR1 /// MIR4640')

# 2. Get a gene mapping dataframe by extracting the ID and Gene Symbol columns
probe_col = 'ID'
gene_col = 'Gene Symbol'
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply the gene mapping to convert probe-level data to gene expression data
# This function handles the division of expression values among mapped genes
# and summing values for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the mapped gene expression data
print("First 10 genes after mapping:")
print(gene_data.index[:10])
print(f"Shape of gene expression data: {gene_data.shape}")


First 10 genes after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')
Shape of gene expression data: (21278, 31)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (19845, 31)


Normalized gene data saved to ../../output/preprocess/Alopecia/gene_data/GSE80342.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM2124815': [0.0, 43.0, 1.0], 'GSM2124816': [0.0, 27.0, 0.0], 'GSM2124817': [0.0, 40.0, 0.0], 'GSM2124818': [1.0, 36.0, 0.0], 'GSM2124819': [1.0, 45.0, 0.0], 'GSM2124820': [1.0, 48.0, 1.0], 'GSM2124821': [1.0, 34.0, 1.0], 'GSM2124822': [1.0, 34.0, 1.0], 'GSM2124823': [1.0, 58.0, 0.0], 'GSM2124824': [1.0, 35.0, 0.0], 'GSM2124825': [1.0, 31.0, 0.0], 'GSM2124826': [1.0, 63.0, 1.0], 'GSM2124827': [1.0, 60.0, 0.0], 'GSM2124828': [1.0, 62.0, 0.0], 'GSM2124829': [1.0, 20.0, 1.0], 'GSM2124830': [1.0, 60.0, 0.0], 'GSM2124831': [1.0, 58.0, 0.0], 'GSM2124832': [1.0, 35.0, 0.0], 'GSM2124833': [1.0, 31.0, 0.0], 'GSM2124834': [1.0, 48.0, 1.0], 'GSM2124835': [1.0, 34.0, 1.0], 'GSM2124836': [1.0, 36.0, 0.0], 'GSM2124837': [1.0, 45.0, 0.0], 'GSM2124838': [1.0, 48.0, 1.0], 'GSM2124839': [1.0, 34.0, 1.0], 'GSM2124840': [1.0, 

Linked data shape after handling missing values: (31, 19848)
Checking for bias in trait distribution...
For the feature 'Alopecia', the least common label is '0.0' with 3 occurrences. This represents 9.68% of the dataset.
The distribution of the feature 'Alopecia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 45.0
  75%: 58.0
Min: 20.0
Max: 63.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 11 occurrences. This represents 35.48% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset usability: False
Dataset is not usable for trait-gene association studies due to bias or other issues.
